# Example-07:   Sky Brightness evolution

Using sky flats taken we determine the sky brightness evolution during the twilight.

<pre>
Máster en Astrofísica UCM  -- Técnicas Experimentales en Astrofísica  
Jaime Zamorano, Nicolás Cardiel and Sergio Pascual

Version 1.0 2021/02/03  
</pre>

In [ ]:
from pylab import *
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.io import fits
from astropy import units as u
from astropy.nddata import CCDData
#import ccdproc

In [ ]:
plt.style.use('./tea.mplstyle')   # Some parameters for nicer graphs

### Working with images in a directory
We will use the files of the first night of observations at NOT (Nordic Optical Telescope) 2008 that can be downloaded from   ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/N1/
or 
http://guaix.fis.ucm.es/~jaz/master_TEA/observaciones_NOT_2008/N1/

``directory`` should point to the working directory.  

We are asumming the BIAS corrected files are located in that directory.


In [ ]:
directory = '/Users/jzamorano/Desktop/NOT_2008/N1/2_corregidos_bias/'   # example

The BIAS corrected files should be located in that directory.
Let\'s create a list containing all the FITS files in that directory.  
We asume that the images were already trimmed and their names begin with 'zt_'

In [ ]:
import os
from glob import glob
# os.path.join is a platform-independent way to join two directories
globpath = os.path.join(directory, 'zt_*.fits')
print(globpath)
# glob searches through directories similar to the Unix shell
filelist = sorted(glob(globpath))
print(filelist[0:5])    # printing only from 0 to 5

### Selecting FLATS files 

With the information in the logbook and after inspecting the files we can prepare lists os FLATS for each filter. We will use filter #76 for the example

In [ ]:
FF_list_76 = ['120057' , '120058' , '120059' 
            , '120060' , '120061' , '120062']

In [ ]:
for i in range(len(FF_list_76)):
    HDUList_object = fits.open(directory+"zt_ALrd"+FF_list_76[i]+".fits")
    primary_header = HDUList_object[0].header
    print(primary_header['FILENAME'],primary_header['OBJECT'],primary_header['exptime']
          ,primary_header['ALFLTID'],primary_header['FBFLTID'],primary_header['DATE-OBS'])

### Statistics and display

In [ ]:
image_flats = []
for file in list_of_flats:
    image_flats.append(CCDData.read(directory+file)) #, unit="adu"))

In [ ]:
# auxiliary function to display a rectangle and compute mean value within it
def draw_rectangle(ax, image_data, x1, x2, y1, y2, color, text=False):
    ax.plot((x1, x1), (y1, y2), color, lw=1)
    ax.plot((x2, x2), (y1, y2), color, lw=1)
    ax.plot((x1, x2), (y1, y1), color, lw=1)
    ax.plot((x1, x2), (y2, y2), color, lw=1)
    if text:
        media = image_data[y1:y2,x1:x2].mean()
        std   = image_data[y1:y2,x1:x2].std()
        ax.text((x1+x2)/2, y1+(y2-y1)/8, str(int(media)), 
                ha='center', va='center', color=color, fontsize=12)        
        ax.text((x1+x2)/2, y2-(y2-y1)/8, str(round(std,1)), 
                ha='center', va='top', color=color, fontsize=12)
    return media, std

In [ ]:
vmin,vmax = 20000,30000
media , s = [],[]
i = 0
fig = plt.figure(figsize=(12, 12))
for i in range(len(FF_list_76)):
        ax = plt.subplot(3,3,i+1)
        img = ax.imshow(image_flats_76[i], cmap='gray', origin='low',vmin=vmin,vmax=vmax)
        #ax.set_xlabel('X axis')
        #ax.set_ylabel('Y axis')
        ax.set_xticks([])
        ax.set_yticks([])
        m,std = draw_rectangle(ax, image_flats_76[i].data, 600, 1200, 600, 1200, color='k',text=True)
        media.append(m)
        s.append(std)
        ax.text(800, 50, image_flats_76[i].header['FILENAME'][7:-5], fontsize=12, color='w')
        ax.text(400,1800,'FLAT filter 76', fontsize=12, color='y')
        ax.text(200, 1550, image_flats_76[i].header['DATE-OBS'], fontsize=12, color='k')
        ax.text(350, 300, 'exposure= '+ str(image_flats_76[i].header['exptime']) +'s', fontsize=12, color='k')
        
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        fig.colorbar(img, cax=cax) #, label='Number of counts')
        ax.grid()
        i = i + 1

In [ ]:
print(media,s)

### Sky brightness graph

In [ ]:
# date observation
fecha , mag = [] , [] 
for i in range(len(FF_list_76)):
    fecha_str = image_flats_76[i].header['DATE-OBS']
    fecha.append(datetime.datetime.strptime(fecha_str, '%Y-%m-%dT%H:%M:%S'))
    counts = np.divide(media[i],image_flats_76[i].header['exptime'])
    log_counts = np.log10(counts)
    magnitude = np.multiply(2.5,log_counts)
    mag.append(magnitude)
print(fecha)

# Using first value as starting point for magnitude differences
mag = np.subtract(mag[0],mag)

In [ ]:
import matplotlib.dates as mdates
plt.figure(figsize=(12,6))
ax0 = plt.subplot(111)
plt.plot(fecha,mag,'ro')
hfmt = mdates.DateFormatter('%H:%M')
ax0.xaxis.set_major_formatter(hfmt)
ax0.set_xlabel('Time')
ax0.set_ylabel('$\Delta$m [mag/arcsec$^2$]')
plt.grid()